<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Project: Data Understanding vacancy from HeadHunter
   

In [22]:
# импорт необходимых библиотек
import pandas as pd # для работы с данными
import psycopg2 # для подключения к Postgresql
import lxml # для работы с ссылками
import requests # для получения данных с сайта
import warnings # для исключения вывода UserWarning в ответах
warnings.filterwarnings("ignore", category=UserWarning) 

In [23]:
# данные для подключения
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Предварительный анализ данных

1. Количество вакансий базе. 

In [24]:
query_3_1 = f'''select count(id)||' вакансий есть в базе' str
           from vacancies
        '''
df = pd.read_sql_query(query_3_1, connection)
print(df['str'][0])

49197 вакансий есть в базе


2. Количество работодателей. 

In [25]:
query_3_2 = f'''select count(id)||' работодателей в базе' str
           from EMPLOYERS
        '''
df = pd.read_sql_query(query_3_2, connection)
print(df['str'][0])

23501 работодателей в базе


3. Количество регионов.

In [26]:
query_3_3 = f'''select count(id)||' регионов в базе' str
           from areas
        '''
df = pd.read_sql_query(query_3_3, connection)
print(df['str'][0])

1362 регионов в базе


4. Посчитать с помощью запроса количество сфер деятельности в базе (таблица industries).

In [27]:
query_3_4 = f'''select count(id)||' сфер деятельности в базе' str
           from industries
        '''
df = pd.read_sql_query(query_3_4, connection)
print(df['str'][0])

294 сфер деятельности в базе


***

In [28]:
print(round(49197/23501, 1), ' вакансий в среденем на одного работодателя')
print(round(49197/1362, 1), ' вакансий в среденем на каждый регион')
print(round(49197/1362, 1), ' вакансий в среденем на каждый регион')

2.1  вакансий в среденем на одного работодателя
36.1  вакансий в среденем на каждый регион
36.1  вакансий в среденем на каждый регион


### Выводы по предварительному анализу данных:

В анализируемой базе данных:
49197 вакансий, 23501 работодателей, 1362 регионов, 294 сфер деятельности.

По предварительным грубым подсчётам можно сказать, что в среднем каждый работодатель публикует две вакансии, а в каждом регионе около 36 вакансий. 
Но нужно учитывать населённость каждого региона и его экономическое развитие, чтобы точнее вычислить средние показатели по каждому региону.



# Детальный анализ вакансий

1. Cколько (cnt) вакансий в каждом регионе (area).

In [29]:
query_4_1 = f'''select a.name area, count(v.id) cnt
           from areas a
           join vacancies v on a.id = v.area_id
           group by area
           order by cnt DESC
           limit 5
        '''
df = pd.read_sql_query(query_4_1, connection)
print(df.to_string(index=False))

           area  cnt
         Москва 5333
Санкт-Петербург 2851
          Минск 2112
    Новосибирск 2006
         Алматы 1892


2. Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой.

In [30]:
query_4_2 = f'''select count(id)||' вакансий, где заполнено хотя бы одно из двух полей с зарплатой.' str
           from  vacancies
           where salary_from is not null or salary_to is not null
        '''
df = pd.read_sql_query(query_4_2, connection)
print(df['str'][0])

24073 вакансий, где заполнено хотя бы одно из двух полей с зарплатой.


3.Cредние значения для нижней и верхней границы зарплатной вилки.

In [31]:
query_4_3 = f'''select avg(salary_from) as from, 
           avg(salary_to) as to
           from  vacancies
        '''
df = pd.read_sql_query(query_4_3, connection)
print(f"{round(df['from'][0])}  нижняя граница, {round(df['to'][0])} верхняя граница по зарплате.")

71065  нижняя граница, 110537 верхняя граница по зарплате.


4. Количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.


In [32]:
query_4_4 = f'''select v1.schedule, v1.employment, count(v1.employment) count
           from  vacancies v1
           group by v1.schedule, v1.employment
           order by count desc
        '''
df = pd.read_sql_query(query_4_4, connection)
print('Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства:')
print(df.head())

Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства:
           schedule           employment  count
0       Полный день     Полная занятость  35367
1  Удаленная работа     Полная занятость   7802
2     Гибкий график     Полная занятость   1593
3  Удаленная работа  Частичная занятость   1312
4    Сменный график     Полная занятость    940


5. Значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [33]:
query_4_5 = f'''select v1.experience, count(v1.id) count
           from  vacancies v1
           group by v1.experience
           order by count
        '''
df = pd.read_sql_query(query_4_5, connection)
print(df.head())

           experience  count
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


***

### Проведя более детальный анализ данных, можно сказать следующее:
- Больше всего вакансий приходятся на города: Москва, Санкт-Петербург, Минск, что не удивительно, потому что Москва и СПб являются самыми населёнными городами России, а также экономическими центрами страны, как и Минск столица и крупнейший город Республики Беларусь.
- Интересно также то, что только примерно у половины вакансий указаны зарплаты. Можно предположить, что работодатели не хотят заранее отталкивать потенциальных работников низкой зарплатой. Или же зарплата на этих местах сдельная и сильно варьируется. Также работодатель может проверить, насколько соискатель заинтересован в данной вакансии, несмотря на зарплату.
- Средний разброс заработной платы 71065 - 110537 рублей, что намного выше средней зарплаты по России. Исходя из собственного опыта по поиску работы, могу предположить, что на hh.ru выкладывают вакансии с высокими требованиями и высокими должностями, где требуется определённое образование и навыки, за которые работодатель готов много платить. На других же сайтах (например, зарплата.ру, работа.ру) выкладывают вакансии с меньшими требованиями и соответственно меньшей зарплатой.
- По занятости можно сказать, что больше работодатели ищут работников с полной занятостью и полным рабочим днём или удалнной работой. Компании важно, чтобы работник находился непосредственно на месте работы, и это была его основное место, а не подработка.
- Больше всего работодатели требуют опыт работы от года до трёх и то трёх до шести лет. А вот с более шестилетним опытом работы вакансий меньше всего. Можно предположить, что и соискателей с шестилетним опытом меньше всего, как и должностей. Скорее всего, это высокие важные должности, которых в любой компании меньше, чем низкооплачиваемых. Также возможно, что при большом опыте работы работодателю придётся долго переучивать нового сотрудника, потому что он уже привык за шесть лет на старой работе делать свою работу определённым образом, который может быть уже не актуален или просто неподходящим для нового работодателя.

# Юнит 5. Анализ работодателей

1. Какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [34]:
query_5_1 = f'''select e.name "Работодатель", count(v.id) "Количество вакансий"
           from VACANCIES v 
           join EMPLOYERS e on e.id = v.employer_id
           group by "Работодатель"
           order by "Количество вакансий" desc
        '''
df = pd.read_sql_query(query_5_1, connection)
print(df.head())
print(f" Первое место - {df['Работодатель'][0]}, пятое место - {df['Работодатель'][4]}")
           

    Работодатель  Количество вакансий
0         Яндекс                 1933
1     Ростелеком                  491
2       Тинькофф                  444
3           СБЕР                  428
4  Газпром нефть                  331
 Первое место - Яндекс, пятое место - Газпром нефть


2. Количество работодателей и вакансий для каждого региона .
Среди регионов, в которых нет вакансий, найти тот, в котором наибольшее количество работодателей.


In [35]:
query_5_2 = f'''select a.name "Регион", count(e.id) "Количество работодателей", count(v.id) "Количество вакансий"
           from  areas a
           full outer join VACANCIES v on v.area_id = a.id
           full outer join EMPLOYERS e on a.id = e.area
           where v.id is null
           group by "Регион"
           order by "Количество работодателей" desc
        '''
df = pd.read_sql_query(query_5_2, connection)
print(df.head())
           

               Регион  Количество работодателей  Количество вакансий
0              Россия                       410                    0
1           Казахстан                       207                    0
2  Московская область                        75                    0
3  Краснодарский край                        19                    0
4            Беларусь                        18                    0


3. Для каждого работодателя вывести количество регионов, в которых он публикует свои вакансии.


In [36]:
query_5_3 = f'''select e.name "Работодатель", count(distinct a.name) "Количество регионов"
           
           from  EMPLOYERS e
           join VACANCIES v on v.employer_id = e.id
           join areas a on a.id = v.area_id
           group by "Работодатель"
           order by "Количество регионов" desc
        '''
df = pd.read_sql_query(query_5_3, connection)
print(df.head())

             Работодатель  Количество регионов
0                  Яндекс                  181
1              Ростелеком                  152
2              Спецремонт                  116
3  Поляков Денис Иванович                   88
4               ООО ЕФИН                    71


4. Количество работодателей, у которых не указана сфера деятельности. 

In [37]:
query_5_4 = f'''select count(e.id) str
from EMPLOYERS_INDUSTRIES ei
full outer join employers e on e.id = ei.employer_id
where ei.employer_id is null
        '''
df = pd.read_sql_query(query_5_4, connection)
print(df['str'][0], 'работодателей, у которых не указана сфера деятельности')

8419 работодателей, у которых не указана сфера деятельности


5. Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [38]:
query_5_5 = f'''select e.name "Работодатель с четырьмя сферами"
from EMPLOYERS_INDUSTRIES ei
full outer join employers e on e.id = ei.employer_id
group by "Работодатель с четырьмя сферами"
having count(ei.industry_id) = 4
order by "Работодатель с четырьмя сферами"
limit 1 offset 2
        '''
df = pd.read_sql_query(query_5_5, connection)
print(df)

  Работодатель с четырьмя сферами
0                            2ГИС


6. У какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [39]:
query_5_6 = f'''select count(e.id) str
from EMPLOYERS_INDUSTRIES ei
full outer join employers e on e.id = ei.employer_id
full outer join industries i on i.id = ei.industry_id
where i.name = 'Разработка программного обеспечения'

        '''
df = pd.read_sql_query(query_5_6, connection)
print(df['str'][0], 'работодателей в сфере деятельности "Разработка программного обеспечения"')

3553 работодателей в сфере деятельности "Разработка программного обеспечения"


7. Для компании «Яндекс» вывести список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавить строку Total с общим количеством вакансий компании. 

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 


In [40]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # ссылка на таблицу городов-миллионников
million_city = pd.read_html(url)[1] # выбираем вторую таблицу на странице
million_city = list(million_city['Город']) # получаем список городов-миллионников
million_city[13] = 'Воронеж' # редактируем название города Воронеж
million_city = tuple(million_city) # кортеж, чтобы можно было вставить в запрос

In [41]:
query_5_7 = f'''with table1 as
(select a.name "Город", count(v.id) as "Количество"
from vacancies v
join areas a on a.id = v.area_id
join employers e on e.id = v.employer_id
where e.name = 'Яндекс' and a.name in {million_city}
group by "Город")

select "Город", "Количество"
from table1
union
select 'Total' as "Город", sum("Количество") as "Количество"
from table1
order by "Количество"
        '''
df = pd.read_sql_query(query_5_7, connection)
print(df)

              Город  Количество
0              Омск        21.0
1         Челябинск        22.0
2        Красноярск        23.0
3         Волгоград        24.0
4             Пермь        25.0
5            Казань        25.0
6    Ростов-на-Дону        25.0
7               Уфа        26.0
8            Самара        26.0
9         Краснодар        30.0
10          Воронеж        32.0
11      Новосибирск        35.0
12  Нижний Новгород        36.0
13     Екатеринбург        39.0
14  Санкт-Петербург        42.0
15           Москва        54.0
16            Total       485.0


***

### Проведя анализ работодателей, можно наблюдать следующее:
- Больше всего вакансий публикуют Яндекс, Ростелеком, Тинькофф, СБЕР, что вполне закономерно, потому что это одни из крупнейших компаний в России вцелом, но также они являются и IT компаниями - даже Сбер и Тинькофф можно считать IT, учитывая их развитие. В последние годы государство России вкладывает большие средства в развитие It-технологий в стране, всвязи с чем и наблюдается большое количество вакансий.
- Вакансий нет в странах и регионах, что тоже логично, потому что обычно в вакансиях указывают город.
- Среди количества регионов по вакансиям наблюдается подозрительная запись - Поляков Денис Иванович. Вряд ли один ИП может выложить вакансии в большем количестве регионов, чем Сбер или Тинькофф. Но возможно этот ИП ищет сотрудников на удалённую работу, поэтому ему не важно местонахождение потенциального работника, поэтому и выкладывает вакансии, в любых регионах.
- У 8419 работодателей не указана сфера деятельности. Затрудняюсь придумать более менее достоверное объяснение, но либо это компании с очень широкой сферой деятельности, что невозможно описать её одним значением, либо они скрывают свою исинную деятельность.
- По городам-миллионникам у компании Яндекс наблюдается та же картина, что и среди всех вакансий - больше всего в крупнейших городах: Москва, Санкт-Петербург, Екатеринбург  

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным.

Вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

In [42]:
query_6_1 = f'''select count(name)||' вакансий' str
from VACANCIES
where (lower(name) like '%data%') or (lower(name) like '%данн%')
        '''
df = pd.read_sql_query(query_6_1, connection)
print(df['str'][0])

1771 вакансий


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста.

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [43]:
query_6_2 = f'''
select count(name)||' вакансия' str
from VACANCIES
where ((name ilike '%data scientist%') or 
(name ilike '%data science%') or
(name ilike '%исследователь данных%')  or
(name ilike '%machine learning%') or
(name ilike '%машинн%обучен%') or
((name) not like '%HTML%' and (name) like '%ML%'))
and (name ilike '%junior%' or experience ilike '%нет опыта%' or employment ilike '%стажировка%')
        '''
df = pd.read_sql_query(query_6_2, connection)
print(df['str'][0])

51 вакансия


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres.

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
query_6_3 = f'''
select count(name)||' вакансия' str
from VACANCIES
where ((name ilike '%data scientist%') or 
(name ilike '%data science%') or
(name ilike '%исследователь данных%')  or
(name ilike '%machine learning%') or
(name ilike '%машинн%обучен%') or
((name) not like '%HTML%' and (name) like '%ML%')) and 
(key_skills ilike '%SQL%' or key_skills ilike '%postgres%')
        '''
df = pd.read_sql_query(query_6_3, connection)
print(df['str'][0])

201 вакансия


4. Насколько популярен Python в требованиях работодателей к DS. Вычислить количество вакансий, в которых в качестве ключевого навыка указан Python.

In [45]:
query_6_4 = f'''
select count(name)||' вакансия' str
from VACANCIES
where ((name ilike '%data scientist%') or 
(name ilike '%data science%') or
(name ilike '%исследователь данных%')  or
(name ilike '%machine learning%') or
(name ilike '%машинн%обучен%') or
((name) not like '%HTML%' and (name) like '%ML%')) and 
(key_skills ilike '%python%')
        '''
df = pd.read_sql_query(query_6_4, connection)
print(df['str'][0])

351 вакансия


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
query_6_5 = f'''
select round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2)||' среднее число навыков' str
from VACANCIES
where ((name ilike '%data scientist%') or 
(name ilike '%data science%') or
(name ilike '%исследователь данных%')  or
(name ilike '%machine learning%') or
(name ilike '%машинн%обучен%') or
((name) not like '%HTML%' and (name) like '%ML%'))
and key_skills is not null

        '''
df = pd.read_sql_query(query_6_5, connection)
print(df['str'][0])

6.41 среднее число навыков


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
query_6_6 = f'''
select distinct experience, round(avg(coalesce(((salary_from + salary_to)/2), salary_from, salary_to))) avg_salary
from VACANCIES
where ((name ilike '%data scientist%') or 
(name ilike '%data science%') or
(name ilike '%исследователь данных%')  or
(name ilike '%machine learning%') or
(name ilike '%машинн%обучен%') or
((name) not like '%HTML%' and (name) like '%ML%'))
and
(salary_from is not null or salary_to is not null)
group by experience
        '''
df = pd.read_sql_query(query_6_6, connection)
print(df)

           experience  avg_salary
0           Нет опыта     74643.0
1  От 1 года до 3 лет    139675.0
2       От 3 до 6 лет    243115.0


***

In [48]:
query = f'''
select distinct experience, count(experience), count(salary_from), count(salary_to)
from VACANCIES
where ((name ilike '%data scientist%') or 
(name ilike '%data science%') or
(name ilike '%исследователь данных%')  or
(name ilike '%machine learning%') or
(name ilike '%машинн%обучен%') or
((name) not like '%HTML%' and (name) like '%ML%'))
group by experience
        '''
df = pd.read_sql_query(query, connection)
print(df)

           experience  count  count  count
0         Более 6 лет     34      0      0
1           Нет опыта     28      5      5
2  От 1 года до 3 лет    223     22     20
3       От 3 до 6 лет    195     19     19


### Согласно предметному анализу:
- Всего 1771 вакансий относятся к работе с данными, среди которых только 51 вакансия подходит для junior'а. закономерно, потому что работодателя выгоднее брать на работу middle или senior. Тем более сейчас достаточно много "джунов", которые сами активно ищут работу после IT курсов, и нет необходимости дополнительно выкладывать вакансии.
- 201 вакансия по SQL, чо на мой взгяд мало, если основной анализ дата саентисты проводят через SQL.
- 351 вакансия, то есть примерно пятая часть всех вакансий, связанных с данными требует знания Python. Это, наверное, вполне нормальное соотношение, учитывая, сколько всего ЯП используются для работы с данными
- Среднее количестов навыков требуют больше шести, что довольно много, ведь аналист должен иметь большой объём знаний и навыков для работы с данными. Но за такую работу и платят соответственно - 75 тысяч без опыта, что почти в полтора раза больше средней зарплаты по России
- Также интересное наблюдение, что для вакансий DS не указывают заработную плату при опыте работы более шести лет, хотя приэтом вакансий при этом условии больше, чем без опыта. Скорее всего З/П с большим опытом не указывают, потому что с таким опытным сотрудником договариваются о З/П уже при собеседовании, чтобы не раскрывать реальные цифры высокопоставленныз специалистов. Так же это работает на конкуренцию - если не указывать З/П у своей компании, то конкурент не узнает, за какую сумма он сможет переманить специалиста. Также З/П у вакансий без опыта указана только у пяти штук. Скорее всего, чтобы не отталкивать неопытных соискателей низкой зарплатой на старте.

### Общий вывод по проекту
- Для снижения нагрузки на вычислительные центрыи и упрощения анализа базы данных, мне кажется, можно удалить регионы с нулевым количестовм вакансий, если это не повредит другим данным.
- Почти триста сфер деятельностей мне кажутся излишними. Можно сократить список на половину, объединив смежные сферы, что также упростит работу с данными.
- Подготовив данные можно будет прогнозировать зарплаты для вакансий, где они не указаны, потому что в данный момент половина вакансий без З/П.
- На мой взгляд, следует детальнее изучить работодателей, которые находятся в топе списка по количеству разных регионов, где они выкладывали свои вакансии. Особенно небольшие частные компании и ИП.
- Так как в столицах - Москва, Санкт-Петербург, Минск - и других крупных городах больше всего вакансий почти при любых условиях фильтрации, то можно рассматривать их отдельно при учёте средних заработных плат и других параметров. Также в столицах можно изучить зависимость количества вакансий, зарплат и прочих показателей в зависимости от района, удалённости от центра и станций метро, добавив эти данные в базу данных.